## Multiple Inheritance and `__init__` Parameter Passing

This notebook explains:
- How `__init__()` works with parameters in multiple inheritance.
- How `super()` uses **Method Resolution Order (MRO)** to decide which class’s constructor runs next.
- How we can explicitly call constructors of parent classes.
- How instance attributes (`self._value`, `self._x`, `self._y`) are shared across constructors because `self` refers to the *same instance* of the final subclass.


In [7]:
class SuperClass2:
    """
    This is the last class in the MRO chain before `object`.

    Responsibilities:
    - Initialize `_x` and `_y` attributes.
    - Demonstrate that `super()` eventually calls `object.__init__()` when the chain ends.
    """

    def __init__(self, x, y):
        print("__init__() in SuperClass2")

        # Call next constructor in the MRO chain (here: object.__init__())
        super().__init__()

        # Initialize class-specific attributes
        self._x = x
        self._y = y
        print(f"SuperClass2 initialized: x = {self._x}, y = {self._y}")

    @staticmethod
    def method_super2():
        print("method_super2() in SuperClass2")

    @staticmethod
    def same_name():
        print("same_name() in SuperClass2")


In [8]:
class SuperClass1:
    """
    SuperClass1 sits before SuperClass2 in the MRO.

    Responsibilities:
    - Initialize `_value`.
    - Pass parameters `x, y` to the next class (SuperClass2).
    """

    def __init__(self, a, x, y):
        # Create attribute in the same instance (not a new object)
        self._value = a
        print("__init__() in SuperClass1")
        print(f"SuperClass1 received: a = {self._value}, x = {x}, y = {y}")

        # This follows MRO: after SuperClass1 → SuperClass2
        super(SuperClass1, self).__init__(x, y) # this line is equal to super().__init() -> Python v3

        print("Returned in SuperClass1 after calling SuperClass2.__init__()")

    @staticmethod
    def method_super1():
        print("method_super1() in SuperClass1")

    @staticmethod
    def same_name():
        print("same_name() in SuperClass1")


In [9]:
class SubClass(SuperClass1, SuperClass2):
    """
    The SubClass inherits from both SuperClass1 and SuperClass2.

    MRO (Method Resolution Order):
        SubClass → SuperClass1 → SuperClass2 → object

    Responsibilities:
    - Start the MRO chain using `super().__init__()`.
    - Also manually call parent constructors to show how explicit calls differ from super().
    """

    def __init__(self, a, x, y):
        print("\n__init__() in SubClass")

        # Starts MRO chain:
        # super() from SubClass goes to SuperClass1
        super().__init__(a, x, y)

        # Explicit calls (not part of MRO)
        SuperClass1.__init__(self, a, x, y)
        SuperClass2.__init__(self, x, y)

        print("Returned to SubClass after explicit constructor calls.")


## **Test Code**

In [10]:
subclass = SubClass(2,3,4)


__init__() in SubClass
__init__() in SuperClass1
SuperClass1 received: a = 2, x = 3, y = 4
__init__() in SuperClass2
SuperClass2 initialized: x = 3, y = 4
Returned in SuperClass1 after calling SuperClass2.__init__()
__init__() in SuperClass1
SuperClass1 received: a = 2, x = 3, y = 4
__init__() in SuperClass2
SuperClass2 initialized: x = 3, y = 4
Returned in SuperClass1 after calling SuperClass2.__init__()
__init__() in SuperClass2
SuperClass2 initialized: x = 3, y = 4
Returned to SubClass after explicit constructor calls.


### **Step-by-step MRO Execution Flow**

1. `SubClass(2, 3, 4)` → calls `SubClass.__init__()`.
2. Inside `SubClass`, `super().__init__(a, x, y)` → MRO says next class is `SuperClass1`.
3. `SuperClass1.__init__()` runs:
   - Creates `_value = a`
   - Calls `super(SuperClass1, self).__init__(x, y)`
4. MRO continues → calls `SuperClass2.__init__()`
   - Creates `_x = x`, `_y = y`
   - Calls `object.__init__()`, which does nothing
5. Control returns back up the chain:
   - Returns to `SuperClass1` → then to `SubClass`
6. Then **explicit calls** to `SuperClass1.__init__()` and `SuperClass2.__init__()` re-run their constructors manually, 
   not following MRO.  
   These use the same `self`, so attributes like `_value`, `_x`, and `_y` are **overwritten**.

---

### Important Concept: Why `self` is shared

- The `self` you pass to each constructor (`SuperClass1.__init__(self, a, x, y)`) is the same object created from `SubClass`.
- This means all attributes (`_value`, `_x`, `_y`) belong to the **same memory instance**, no matter which class initialized them.
- Constructors never create a *new object* — they *configure the one you already have*.

---

### `super()` vs Explicit Call

| Expression | Follows MRO | Behavior |
|-------------|--------------|-----------|
| `super().__init__()` | ✅ Yes | Calls the next class in MRO order |
| `super(SuperClass1, self).__init__()` | ✅ Yes | Continues from `SuperClass1`’s place in MRO |
| `SuperClass1.__init__(self, ...)` | ❌ No | Direct call, independent of MRO |
| `SuperClass2.__init__(self, ...)` | ❌ No | Direct call, runs again if already called via MRO |

---

### 🔚 Final Takeaway

- `__init__` is *just a method*, not a magic one. It sets attributes on the instance (`self`).
- `super()` is the mechanism that lets multiple inheritance cooperate cleanly via **MRO**.
- Explicit constructor calls (`Class.__init__(self)`) break the cooperative chain, so they must be used carefully.